# Dependency set up

Let's download and import necessary libraries
- `geopandas` for Geographic manipulations
- `python-igraph` for Graph processing

In [1]:
!pip install --quiet geopandas python-igraph cairocffi

In [2]:
import os
import requests
from typing import NamedTuple
import shutil
from zipfile import ZipFile

import numpy as np
import pandas as pd
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point, LineString

Set Mapbox for maps awesomness

In [3]:
MAPBOX_ACCESS_TOKEN = ""  # Paste access token
px.set_mapbox_access_token(MAPBOX_ACCESS_TOKEN)

## Data download

We based the stations data between different files in the [Developers Data](http://web.mta.info/developers/developer-data-terms.html#data) page
- the Google GTFS Transit file (stores stops only)
- the Stations CSV that groups stops into stations and Complexes
- the Complexes CSV that stores the station complexes

In [4]:
GTFS_URL = 'http://web.mta.info/developers/data/nyct/subway/google_transit.zip'
STATIONS_CSV = 'https://atisdata.s3.amazonaws.com/Station/Stations.csv'
COMPLEXES_CSV = 'http://web.mta.info/developers/data/nyct/subway/StationComplexes.csv'

def download_file(url, to):
  print(f'Downloading {url} --> {to}')
  with requests.get(url, stream=True) as response:
    response.raise_for_status()
    with open(to, 'wb') as fileobj:
      response.raw.decode_content = True
      shutil.copyfileobj(response.raw, fileobj)

# Download the file
download_file(GTFS_URL, 'google_transit.zip')

# Deflate it to the current directory
with ZipFile('google_transit.zip') as google_transit_zip:
  names = "\n  ".join(google_transit_zip.namelist())
  print(f'Extracting:\n  {names}')
  google_transit_zip.extractall()
  del names, google_transit_zip

# Cleanup all files
os.remove('google_transit.zip')

download_file(STATIONS_CSV,  "stations.csv")
download_file(COMPLEXES_CSV, "complexes.csv")

Extracting:
  agency.txt
  calendar.txt
  calendar_dates.txt
  routes.txt
  shapes.txt
  stops.txt
  stop_times.txt
  transfers.txt
  trips.txt


# Build nodes


## Stops

First we read the stops. I selected only the non null columns.

In [5]:
stops = pd.read_csv('stops.txt', index_col='stop_id')

# Drop columns without any data
stops.dropna(axis=1, how='all', inplace=True)

stops

,stop_name,stop_lat,stop_lon,location_type,parent_station
stop_id,,,,,
101,Van Cortlandt Park-242 St,40.889248,-73.898583,1,NaN
101N,Van Cortlandt Park-242 St,40.889248,-73.898583,0,101
101S,Van Cortlandt Park-242 St,40.889248,-73.898583,0,101
103,238 St,40.884667,-73.900870,1,NaN
103N,238 St,40.884667,-73.900870,0,103
...,...,...,...,...,...
S30N,Tompkinsville,40.636949,-74.074835,0,S30
S30S,Tompkinsville,40.636949,-74.074835,0,S30
S31,St George,40.643748,-74.073643,1,NaN


Based on the GTFS Developer reference: https://developers.google.com/transit/gtfs/reference

`location_type`:

- 0: Stop
- 1: Station
- 2: Entrance/Exit
- 3: Generic location
- 4: Boarding areas

Fill missing parent station for `location_type` 1 with itself:

In [6]:
# Based on: https://stackoverflow.com/a/30357342
stop_id = stops.index.to_series()
stops["parent_station"] = np.where(
    stops["location_type"]==1,
    stop_id,
    stops["parent_station"]
    )

## Complexes

I read the complexes and remove the unnecessary columns related to Accessibility. Complexes are group of different stops that are together in the same area.

In [7]:
complexes = pd.read_csv('complexes.csv', index_col='Complex ID')
complexes.drop(['ADA', 'ADA Notes'], axis=1, inplace=True) # Remove unnecessary columns
complexes

,Complex Name
Complex ID,
601,14 St / 6 Av
602,14 St - Union Sq
603,149 St - Grand Concourse
604,161 St - Yankee Stadium
605,168 St - Washington Hts
606,Court Sq - 23 St
607,34 St - Herald Sq
608,4 Av - 9 St
609,42 St - Bryant Pk / 5 Av


## Stations

Same with stations. This data doesn't come with the GTFS data but it is useful to figure out which GTFS stops are in the same "Station Complex"

In [8]:
stations = pd.read_csv('stations.csv', index_col='GTFS Stop ID')
stations.drop(['ADA', 'ADA Notes'], axis=1, inplace=True) # Remove unnecessary columns
stations

,Station ID,Complex ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
GTFS Stop ID,,,,,,,,,,,,
R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
R03,2,2,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
R04,3,3,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
R05,4,4,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
R06,5,5,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
S15,517,517,SIR,Staten Island,Prince's Bay,SI,SIR,Open Cut,40.525507,-74.200064,St George,Tottenville
S14,518,518,SIR,Staten Island,Pleasant Plains,SI,SIR,Embankment,40.522410,-74.217847,St George,Tottenville
S13,519,519,SIR,Staten Island,Richmond Valley,SI,SIR,Open Cut,40.519631,-74.229141,St George,Tottenville


## Stop Times

In [9]:
stop_times = pd.read_csv('stop_times.txt')

# Drop columns without any data
stop_times.dropna(axis=1, how='all', inplace=True)
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,AFA21GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,0,0
1,AFA21GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,0,0
2,AFA21GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,0,0
3,AFA21GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,0,0
4,AFA21GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,0,0
...,...,...,...,...,...,...,...
542113,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,05:03:00,05:03:00,S27N,17,0,0
542114,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,05:06:00,05:06:00,S28N,18,0,0
542115,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,05:08:00,05:08:00,S29N,19,0,0
542116,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,05:10:00,05:10:00,S30N,20,0,0


## Trips

In [10]:
trips = pd.read_csv('trips.txt', index_col='trip_id')

trips.dropna(axis=1, how='all', inplace=True)

trips

,route_id,service_id,trip_headsign,direction_id,shape_id
trip_id,,,,,
AFA21GEN-1037-Sunday-00_000600_1..S03R,1,AFA21GEN-1037-Sunday-00,South Ferry,1,1..S03R
AFA21GEN-1037-Sunday-00_002600_1..S03R,1,AFA21GEN-1037-Sunday-00,South Ferry,1,1..S03R
AFA21GEN-1037-Sunday-00_004600_1..S03R,1,AFA21GEN-1037-Sunday-00,South Ferry,1,1..S03R
AFA21GEN-1037-Sunday-00_006600_1..S03R,1,AFA21GEN-1037-Sunday-00,South Ferry,1,1..S03R
AFA21GEN-1037-Sunday-00_007200_1..N03R,1,AFA21GEN-1037-Sunday-00,Van Cortlandt Park-242 St,0,1..N03R
...,...,...,...,...,...
SIR-FA2017-SI017-Sunday-00_138600_SI..S03R,SI,SIR-FA2017-SI017-Sunday-00,Tottenville,1,NaN
SIR-FA2017-SI017-Sunday-00_141100_SI..N03R,SI,SIR-FA2017-SI017-Sunday-00,St George,0,NaN
SIR-FA2017-SI017-Sunday-00_141600_SI..S03R,SI,SIR-FA2017-SI017-Sunday-00,Tottenville,1,NaN


## Calendar 
This is important to measure how many times does the trip connect to a station

In [11]:
calendar = pd.read_csv('calendar.txt', index_col='service_id')

# Let's calculate how many times the calendar runs a week
calendar['ndays_week'] = calendar['monday'] + calendar['tuesday'] + \
  calendar['wednesday'] + calendar['thursday'] + calendar['friday'] + \
  calendar['saturday'] + calendar['sunday']

calendar

,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,ndays_week
service_id,,,,,,,,,,
AFA21GEN-1037-Sunday-00,0,0,0,0,0,0,1,20211219,20220619,1
AFA21GEN-2048-Sunday-00,0,0,0,0,0,0,1,20211219,20220619,1
AFA21GEN-3041-Sunday-00,0,0,0,0,0,0,1,20211219,20220619,1
AFA21GEN-4049-Sunday-00,0,0,0,0,0,0,1,20211219,20220619,1
AFA21GEN-5048-Sunday-00,0,0,0,0,0,0,1,20211219,20220619,1
...,...,...,...,...,...,...,...,...,...,...
BFA21GEN-M024-Saturday-00,0,0,0,0,0,1,0,20211224,20220625,1
BFA21GEN-N056-Saturday-00,0,0,0,0,0,1,0,20211224,20220625,1
BFA21GEN-Q021-Saturday-00,0,0,0,0,0,1,0,20211224,20220625,1


# Start doing some Notebook plays



## Calculate how many trains go through each pair of stops

First, join dataframes to combine all columns, we want to know for each stop, 
- How many times it pass per week
- The Station Complex it belongs to

In [12]:
stop_times_joined = (stop_times[['trip_id', 'stop_id', 'stop_sequence']]
  .join(stops['parent_station'], on='stop_id')
  .join(stations['Complex ID'], on='parent_station')
  .join(trips['service_id'], on='trip_id')
  .join(calendar['ndays_week'], on='service_id'))

stop_times_joined

,trip_id,stop_id,stop_sequence,parent_station,Complex ID,service_id,ndays_week
0,AFA21GEN-1037-Sunday-00_000600_1..S03R,101S,1,101,293,AFA21GEN-1037-Sunday-00,1
1,AFA21GEN-1037-Sunday-00_000600_1..S03R,103S,2,103,294,AFA21GEN-1037-Sunday-00,1
2,AFA21GEN-1037-Sunday-00_000600_1..S03R,104S,3,104,295,AFA21GEN-1037-Sunday-00,1
3,AFA21GEN-1037-Sunday-00_000600_1..S03R,106S,4,106,296,AFA21GEN-1037-Sunday-00,1
4,AFA21GEN-1037-Sunday-00_000600_1..S03R,107S,5,107,297,AFA21GEN-1037-Sunday-00,1
...,...,...,...,...,...,...,...
542113,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,S27N,17,S27,505,SIR-FA2017-SI017-Sunday-00,1
542114,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,S28N,18,S28,504,SIR-FA2017-SI017-Sunday-00,1
542115,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,S29N,19,S29,503,SIR-FA2017-SI017-Sunday-00,1
542116,SIR-FA2017-SI017-Sunday-00_027100_SI..N03R,S30N,20,S30,502,SIR-FA2017-SI017-Sunday-00,1


Those are the trains that depature from a complex per week

In [13]:
trains_per_complex = stop_times_joined.groupby('Complex ID').sum()['ndays_week'].to_frame('trains')
trains_per_complex.sort_values('trains')

,trains
Complex ID,
196,604
506,797
509,797
502,797
503,797
...,...
601,13706
610,14638
617,15884


This dataframe has for each row a pair of stops that is connecting. It is much faster than iterating in each row in Pandas

In [14]:
# Set the next trip ID :)
stop_times_connections = stop_times_joined.copy()
stop_times_connections['next_trip_id'] = stop_times['trip_id']
stop_times_connections['next_stop_sequence'] = stop_times['stop_sequence'] + 1

# Find the row with the next stop
stop_times_connections_indexed = stop_times_connections.set_index(['trip_id', 'stop_sequence'])
stop_times_connections = stop_times_connections.join(
    stop_times_connections_indexed[['stop_id', 'parent_station', 'Complex ID']],
    on=['next_trip_id', 'next_stop_sequence'], rsuffix="_next")

# Remove the rows with no next connection, which is the last for each trip
stop_times_connections.drop(['next_trip_id', 'next_stop_sequence'], axis=1, inplace=True)
stop_times_connections = stop_times_connections.loc[stop_times_connections['stop_id_next'].notna()]

# Convert Complex ID_next back to an integer
stop_times_connections = stop_times_connections.astype({'Complex ID_next': np.int64 })

del stop_times_connections_indexed

# Simplify columns a little bit
stop_times_connections.set_index(['trip_id', 'stop_sequence'], inplace=True)
stop_times_connections.drop('service_id', axis=1, inplace=True)
stop_times_connections

# Create the Columns C1 and C2 to group trips by complexes
stop_times_connections['C1'] = stop_times_connections[['Complex ID','Complex ID_next']].min(axis=1)
stop_times_connections['C2'] = stop_times_connections[['Complex ID','Complex ID_next']].max(axis=1)

Aggregate and count trips between complexes in the lapse of a week. Each connection can appear either 1,4 or 5 days a week (Sundays, Weekdays, or Thursdays to Fridays). So each row has to be multiplied by the days of the week it actually runs.

$W_{C_1C_2} = \sum_{r \in Conn(C_1, C_2)} r_{ndays\_week}$

Where

$Conn(C_1,C_2)$ are the rows that have $C_1$ or $C_2$ as Complex ID and Next complex ID, in any order

In [15]:
trips_between_complexes = stop_times_connections.groupby(['C1', 'C2']).sum()
trips_between_complexes.rename(columns={'ndays_week':'trains_week'}, inplace = True)
trips_between_complexes

Complex ID  trains_week  Complex ID_next
C1  C2                                           
1   2          1314         2652             1314
2   3          2190         2652             2190
3   4          3066         2652             3066
4   5          3942         2652             3942
5   6          4818         2652             4818
...             ...          ...              ...
620 635      533980         2167           534025
    636      534452         2167           534404
622 623     1202049         5351          1202046
    628     1375000         5888          1375000
624 628     1731516         7254          1731516

[564 rows x 3 columns]

## Plot the different stations by complex

First idea, let's aggregate (parent) stations into different complexes. Riders don't leave complexes when switching between lines. I believe different GTFS stops in the same complex should belong to the same node in our graph. Given that, I joined all stations into groups of complexes. 

<img width="200" src="https://thumbs.dreamstime.com/b/new-york-subway-signs-may-information-pole-different-lines-fulton-street-82542452.jpg">

One example, is [Fulton Street](https://en.wikipedia.org/wiki/Fulton_Street_station_(New_York_City_Subway)), a massive complex with a shopping mall inside. The image below shows the lines that pass in Fulton street.



In [16]:
joined_stations = stations.reset_index(level=0).join(complexes, on="Complex ID")

def daytime_routes_combination(series):
  unique_values = set()
  for _, value in series.iteritems():
    unique_values |= set(value.split())
  return sorted(unique_values)

def station_group_name(row):
  complex_name = row['Complex Name']
  stop_name = row['Stop Name']
  if isinstance(complex_name, str):
      return complex_name
  elif isinstance(complex_name, np.ndarray) and all((not np.isnan(x) for x in complex_name)):
    return complex_name
  elif isinstance(stop_name, np.ndarray):
    return ', '.join(stop_name)
  else:
    return stop_name
  
  
aggregated_stations = joined_stations.groupby('Complex ID').agg({
    'GTFS Latitude': np.mean,
    'GTFS Longitude': np.mean,
    'Stop Name': np.unique,
    'Complex Name': np.unique,
    'Borough': np.unique,
    'Daytime Routes': daytime_routes_combination,
    'GTFS Stop ID': np.unique,
})

aggregated_stations['Display Name'] = aggregated_stations.apply(station_group_name, axis=1)
aggregated_stations = aggregated_stations.join(trains_per_complex)

del joined_stations

aggregated_stations

,GTFS Latitude,GTFS Longitude,Stop Name,Complex Name,Borough,Daytime Routes,GTFS Stop ID,Display Name,trains
Complex ID,,,,,,,,,
1,40.775036,-73.912034,Astoria-Ditmars Blvd,NaN,Q,"[N, W]",R01,Astoria-Ditmars Blvd,2652
2,40.770258,-73.917843,Astoria Blvd,NaN,Q,"[N, W]",R03,Astoria Blvd,2652
3,40.766779,-73.921479,30 Av,NaN,Q,"[N, W]",R04,30 Av,2652
4,40.761820,-73.925508,Broadway,NaN,Q,"[N, W]",R05,Broadway,2652
5,40.756804,-73.929575,36 Av,NaN,Q,"[N, W]",R06,36 Av,2652
...,...,...,...,...,...,...,...,...,...
628,40.710089,-74.007838,Fulton St,Fulton St,M,"[2, 3, 4, 5, A, C, J, Z]","[229, 418, A38, M22]",Fulton St,13142
629,40.713428,-73.950847,"[Lorimer St, Metropolitan Av]",Metropolitan Av / Lorimer St,Bk,"[G, L]","[G29, L10]",Metropolitan Av / Lorimer St,5538
630,40.699622,-73.911986,Myrtle-Wyckoff Avs,Myrtle - Wyckoff Avs,Bk,"[L, M]","[L17, M08]",Myrtle - Wyckoff Avs,5484


In [17]:
edges_between_complexes = trips_between_complexes.copy()
edges_between_complexes = edges_between_complexes.join(
    aggregated_stations[['GTFS Latitude', 'GTFS Longitude']],
    on='C1')

edges_between_complexes = edges_between_complexes.join(
    aggregated_stations[['GTFS Latitude', 'GTFS Longitude']],
    on='C2', rsuffix='_next')

edges_between_complexes

Complex ID  trains_week  Complex ID_next  GTFS Latitude  \
C1  C2                                                             
1   2          1314         2652             1314      40.775036   
2   3          2190         2652             2190      40.770258   
3   4          3066         2652             3066      40.766779   
4   5          3942         2652             3942      40.761820   
5   6          4818         2652             4818      40.756804   
...             ...          ...              ...            ...   
620 635      533980         2167           534025      40.693241   
    636      534452         2167           534404      40.693241   
622 623     1202049         5351          1202046      40.713154   
    628     1375000         5888          1375000      40.713154   
624 628     1731516         7254          1731516      40.712603   

         GTFS Longitude  GTFS Latitude_next  GTFS Longitude_next  
C1  C2                                                            
1   2        -73.912034           40.770258           -73.917843  
2   3        -73.917843           40.766779           -73.921479  
3   4        -73.921479           40.761820           -73.925508  
4   5        -73.925508           40.756804           -73.929575  
5   6        -73.929575           40.752882           -73.932755  
...                 ...                 ...                  ...  
620 635      -73.990642           40.702577           -74.013329  
    636      -73.990642           40.692259           -73.986642  
622 623      -74.003766           40.718701           -74.000580  
    628      -74.003766           40.710089           -74.007838  
624 628      -74.009552           40.710089           -74.007838  

[564 rows x 7 columns]

In [18]:
def reduce_array(series):
   return [item for sublist in series for item in sublist]

def combine_latitudes(row):
  return [row['GTFS Latitude'], row['GTFS Latitude_next'], None]

def combine_longitudes(row):
  return [row['GTFS Longitude'], row['GTFS Longitude_next'], None]



latitudes = reduce_array(edges_between_complexes.apply(combine_latitudes, axis=1))
longitudes = reduce_array(edges_between_complexes.apply(combine_longitudes, axis=1))

fig_lines = px.line_mapbox(lat=latitudes, lon=longitudes, zoom=10, width=1000, height=800)

del latitudes, longitudes

fig_scatter = px.scatter_mapbox(aggregated_stations, 
                        lat="GTFS Latitude", 
                        lon="GTFS Longitude", 
                        text='Display Name', 
                        hover_data=['Daytime Routes', 'GTFS Stop ID', 'Display Name'],
                        color='trains',
                        size='trains',
                        zoom=10)
fig_lines.add_trace(fig_scatter.data[0])
fig_lines.show()

del fig_scatter, fig_lines

## Build Graph

Combine `aggregated_stations` and `edges_between_stations` to form an 
undirected graph with the trains per station

In [19]:
aggregated_stations.Borough.unique()

array(['Q', 'M', 'Bk', 'Bx', 'SI'], dtype=object)

In [20]:
stations_graph = ig.Graph()

vertexes = [str(complex_id) for complex_id in aggregated_stations.index.tolist()]

def normalize(series):
  return (series - series.min()) / (series.max() - series.min())

# Size is 3 + 20 * s where s is the normalized_value
def vertex_sizes(series):
  return (20 * normalize(series) + 5).values

def label_sizes(series):
  amplitude = series.max() - series.min()
  return (7 * normalize(series) + 6).values

def width(series):
  amplitude = series.max() - series.min()
  return (5 * normalize(series) + 0.5).values

def edge_color(series):
  color_val = (1 - 0.8 * normalize(series)).values
  return [(val, val, val) for val in color_val] # An RGB color tuple

def color(borough):
  if borough == 'Q':
    return 'Orange'
  if borough == 'M':
    return 'Crimson'
  if borough == 'Bk':
    return 'Steel Blue'
  if borough == 'Bx':
    return 'Web Green'
  if borough == 'SI':
    return 'Light Cyan'

stations_graph.add_vertices(vertexes, attributes={
    'trains': aggregated_stations.trains.values,
    'size': vertex_sizes(aggregated_stations.trains),
    'daytime_routes': aggregated_stations['Daytime Routes'].values,
    'color': aggregated_stations.Borough.apply(color).values,
})


edges = ((str(u), str(v)) for (u,v) in edges_between_complexes.index.tolist())

stations_graph.add_edges(edges, attributes={
    'trains': edges_between_complexes.trains_week.values,
})


del vertexes, edges

print(stations_graph.summary())


IGRAPH UN-- 445 564 -- 
+ attr: color (v), daytime_routes (v), name (v), size (v), trains (v), trains (e)


### Compute a Layout
First we issue the vertex positions according to they geographical location. 
It is important to start from a position where edges do not overlap (too much) 
so that we get some a decent layout. By default Fruchterman Reingold uses a 
random layout and that causes a chaotic layour

Note: If we set the paramter `niter=0` we get the positions as they are in the map. Also, do NOT use weights in the layout because if attract larger nodes too much. This one spaciates nodes a little bit


In [21]:

node_initial_positions = list(
    zip(normalize(aggregated_stations['GTFS Longitude'].values), 
        normalize(-aggregated_stations['GTFS Latitude'].values)))

# This part can slow
# If we set niter=0 we get the positions as they are in a map
# Also, do NOT use weights in the layout because if attract larger nodes too much
# This one spaciates nodes a little bit
layout = stations_graph.layout_fruchterman_reingold(
    seed=node_initial_positions,
    niter=500)

print('Done calculating layout')

Done calculating layout


In [22]:
plot = ig.plot(stations_graph,
        target='stations_graph.png',
        autocurve=True,
        vertex_label=aggregated_stations['Display Name'].values,
        vertex_label_size=label_sizes(aggregated_stations.trains),
        edge_color=edge_color(edges_between_complexes.trains_week),
        edge_width=width(edges_between_complexes.trains_week),
        vertex_label_dist=1,
        margin=50,
        bbox=(0, 0, 950, 600),
        vertex_frame_width=0.5,
        layout=layout)
plot

Save the file into GML format

In [23]:
stations_graph.vs["latitude"] = aggregated_stations["GTFS Latitude"].values
stations_graph.vs["longitude"] = aggregated_stations["GTFS Longitude"].values
stations_graph.vs["longitude"] = aggregated_stations["GTFS Longitude"].values
stations_graph.vs["display_name"] = aggregated_stations["Display Name"].values

In [24]:
stations_graph.write("../data/nyc_subway.gml", format="gml")

c:\Users\rauls\Anaconda3\envs\nyc\lib\site-packages\igraph\__init__.py:2988: RuntimeWarning:

A non-numeric, non-string, non-boolean edge attribute was ignored at src/io/gml.c:717



# Transfers (not important)

Those are the transfers that connect different routes together. Initially I thought those were important to determine if two stops are connected, but nah, using Stations are Complexes is far more useful

In [25]:
transfers = pd.read_csv('transfers.txt')
transfers = transfers.loc[transfers['from_stop_id'] < transfers['to_stop_id']]
transfers

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
10,112,A09,2,180
24,125,A24,2,180
27,127,725,2,180
28,127,902,2,180
29,127,A27,2,300
...,...,...,...,...
482,L03,R20,2,180
495,L17,M08,2,180
527,M20,Q01,2,180
528,M20,R23,2,300


Those are the transfers that connect routes between different complexes:
None of those two are neither important, so we can ignore them
- Junius St and Livonia Av (you still do it on foot according to the [news](https://www.amny.com/transit/mta-to-connect-two-brooklyn-stations-and-extend-free-transfer-program/))
- Lexington Av / 59 St and Lexington Av/63 St. Those are four blocks of distance apart, so you have to make the commute on foot, and it is not a super important station, well, if you come from Queens maybe.

In [26]:
# First parent stops
parent_stops = stops.loc[stops['parent_station'].isna()]
parent_stops = parent_stops.join(stations).join(complexes, on="Complex ID")

# Then transfers for stations in different complexes.
joined_transfers = (transfers.set_index('from_stop_id')
  .join(parent_stops[['Complex ID', 'Complex Name', 'Stop Name']])
  .reset_index(level=0)
  .set_index('to_stop_id')
  .join(parent_stops[['Complex ID', 'Complex Name', 'Stop Name']], rsuffix='2')
  .reset_index(level=0))
joined_transfers.loc[joined_transfers['Complex ID'] != joined_transfers['Complex ID2']]

,to_stop_id,from_stop_id,transfer_type,min_transfer_time,Complex ID,Complex Name,Stop Name,Complex ID2,Complex Name2,Stop Name2
0,A09,112,2,180,NaN,NaN,NaN,NaN,NaN,NaN
1,A24,125,2,180,NaN,NaN,NaN,NaN,NaN,NaN
2,725,127,2,180,NaN,NaN,NaN,NaN,NaN,NaN
3,902,127,2,180,NaN,NaN,NaN,NaN,NaN,NaN
4,A27,127,2,300,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
71,R20,L03,2,180,NaN,NaN,NaN,NaN,NaN,NaN
72,M08,L17,2,180,NaN,NaN,NaN,NaN,NaN,NaN
73,Q01,M20,2,180,NaN,NaN,NaN,NaN,NaN,NaN
74,R23,M20,2,300,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
del joined_transfers, parent_stops